In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from sch.models import *
import datetime as dt
from django.db.models import CharField
from sch.actions import *
from sch.xviews import week

In [2]:
from sch.actions import *
from sch.xviews import week

In [3]:
week.get_days("2022-W42")

<WorkdayManager [<Workday: 2022 10 16>, <Workday: 2022 10 17>, <Workday: 2022 10 18>, <Workday: 2022 10 19>, <Workday: 2022 10 20>, <Workday: 2022 10 21>, <Workday: 2022 10 22>]>

In [6]:
emps = Employee.objects.in_other_slot(workday=Workday.objects.get(slug='2022-10-31'),shift=Shift.objects.get(name__contains='7C'))
emps = emps.filter(shifts_trained=Shift.objects.get(name__contains='7C'))
emps

<EmployeeManager [<Employee: Brianna>, <Employee: Elisia>, <Employee: Jaden>, <Employee: Trisha>, <Employee: Danica>, <Employee: Nicki>]>

In [7]:
shiftsPerWeek = 0
for sft in Shift.objects.filter(start__hour__gte=10, cls='CPhT').values_list('occur_days',flat=True):
    shiftsPerWeek += len(sft)
shiftsPerWeek

28

## NIGHT PERCENTAGE OF TOTAL SHIFTS"

In [8]:
NIGHT_PERCENTS = []
for emp in Employee.objects.all():
    if Slot.objects.filter(employee=emp).exists():
        perc = round(Slot.objects.filter(employee=emp,shift__start__hour__gte=10).count()/Slot.objects.filter(employee=emp).count()*100, 1)
        NIGHT_PERCENTS += [f"{emp}: {perc}% EVENINGS  {int(perc/4) *'+'}"]
        
NIGHT_PERCENTS
        

['Josh: 3.2% EVENINGS  ',
 'Sabrina: 45.0% EVENINGS  +++++++++++',
 'Brianna: 0.0% EVENINGS  ',
 'Molly: 23.5% EVENINGS  +++++',
 'Doug: 20.5% EVENINGS  +++++',
 'Brittanie: 0.0% EVENINGS  ',
 'Esperanza: 4.3% EVENINGS  +',
 'Carrie: 31.0% EVENINGS  +++++++',
 'Elisia: 76.7% EVENINGS  +++++++++++++++++++',
 'Jaden: 22.0% EVENINGS  +++++',
 'Lynlee: 28.2% EVENINGS  +++++++',
 'Carol: 62.7% EVENINGS  +++++++++++++++',
 'Trisha: 31.0% EVENINGS  +++++++',
 'Danica: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Tiffany: 27.2% EVENINGS  ++++++',
 'Brianna S: 0.0% EVENINGS  ',
 'Amanda: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Michael: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Mona: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Mary: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Mark B: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Andrew: 53.2% EVENINGS  +++++++++++++',
 'Mei: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'Tony: 100.0% EVENINGS  +++++++++++++++++++++++++',
 'S

## 2022 NOV 9 : TECH SCHEDULE

In [9]:
today = Workday.objects.get(date=dt.date.today())
today_slots = Slot.objects.filter(workday=today).order_by('shift__start')
for slot in today_slots:
    if slot.employee.cls == "CPhT":
        print (slot)

221120-t-MI Nicki
221120-t-7C David
221120-t-7P Esperanza
221120-t-EI Sabrina
221120-t-EP Cheryl
221120-t-3 Mona
221120-t-N Amanda


## SLOTS FROM A WEEK

In [10]:
weekWds = today.siblings_iweek
weekSlots = Slot.objects.filter(workday__in=weekWds).order_by('-employee')
for slot in weekSlots[:20]:
    print(slot)
print("....")

221120-p-S Corwin
221120-t-EP Cheryl
221122-t-MI Cheryl
221124-t-MI Cheryl
221125-t-EP Cheryl
221120-p-C Kat
221122-p-E Kat
221124-p-M Kat
221125-p-S Kat
221120-p-M Alex
221121-p-C Alex
221122-p-PC Alex
221124-p-S Alex
221121-p-M Shaun
221122-p-I Shaun
221123-p-C Shaun
221125-p-C Shaun
221122-p-C Kristen
221123-p-PC Kristen
221126-p-S Kristen
....


In [11]:
def getTradableSlots(slot):
    print('INPUT: ', slot)
    emp = slot.employee
    sft = slot.shift
    wd = slot.workday
    weekdays = slot.workday.siblings_iweek
    daysOn = Slot.objects.filter(workday__in = weekdays,employee = emp).values('workday').distinct()
    daysOff = weekdays.exclude(pk__in=daysOn)
    print('daysOff:',list(daysOff.values_list('date__day',flat=True)))
    tenable = []
    slotDaysOff = Slot.objects.filter(workday__in=daysOff, shift__in=emp.shifts_available.all()).order_by('workday')
    print(slotDaysOff)
    for slotB in slotDaysOff:
        if emp in slot.fillableBy():
            if slotB.employee in slot.fillableBy():
                tenable.append(slotB)
    print('tenableTrades:',len(tenable),[f'{i.shift}>{i.workday.date.day}' for i in tenable])
    return tenable



In [12]:
slots = Slot.objects.filter(employee__name="Trisha",workday__iweek=43,workday__date__year=2022)
for slot in slots:
    print('-------------')
    print(slot.is_turnaround)
    getTradableSlots(slot)

-------------
0
INPUT:  221025-t-OP Trisha
daysOff: [23, 28, 29]
<SlotManager [<Slot: 221023-t-MI Cheryl>, <Slot: 221023-t-7C Josh>, <Slot: 221023-t-7P Esperanza>, <Slot: 221023-t-EI Nicki>, <Slot: 221023-t-EP David>, <Slot: 221023-t-N Amanda>, <Slot: 221028-t-MI David>, <Slot: 221028-t-S Josh>, <Slot: 221028-t-7C Tiffany>, <Slot: 221028-t-7P Brianna>, <Slot: 221028-t-OP Brittanie>, <Slot: 221028-t-EI Elisia>, <Slot: 221028-t-EP Jaden>, <Slot: 221028-t-N Michael>, <Slot: 221029-t-MI Tiffany>, <Slot: 221029-t-7C Jaden>, <Slot: 221029-t-7P Brianna>, <Slot: 221029-t-EI Sabrina>, <Slot: 221029-t-EP Elisia>, <Slot: 221029-t-N Michael>]>
tenableTrades: 4 ['t-7C>23', 't-S>28', 't-OP>28', 't-EI>29']
-------------
0
INPUT:  221026-t-OP Trisha
daysOff: [23, 28, 29]
<SlotManager [<Slot: 221023-t-MI Cheryl>, <Slot: 221023-t-7C Josh>, <Slot: 221023-t-7P Esperanza>, <Slot: 221023-t-EI Nicki>, <Slot: 221023-t-EP David>, <Slot: 221023-t-N Amanda>, <Slot: 221028-t-MI David>, <Slot: 221028-t-S Josh>, <S

In [13]:
wds = Workday.objects.filter(date__year=2022,ischedule=7)
ss = Slot.objects.filter(workday__in=wds)
ss

<SlotManager [<Slot: 221016-t-MI Tiffany>, <Slot: 221016-p-I Lynlee>, <Slot: 221016-t-7C Trisha>, <Slot: 221016-t-7P Brianna>, <Slot: 221016-p-C Carrie>, <Slot: 221016-t-EI Jaden>, <Slot: 221016-t-EP Elisia>, <Slot: 221016-p-3 Andrew>, <Slot: 221016-t-N Michael>, <Slot: 221016-p-N Mary>, <Slot: 221016-p-S Kristen>, <Slot: 221016-t-3 Danica>, <Slot: 221016-p-M Sam>, <Slot: 221017-t-7P Brianna>, <Slot: 221017-t-OP Brittanie>, <Slot: 221017-p-MO Brianna S>, <Slot: 221017-t-N Michael>, <Slot: 221017-p-N Mary>, <Slot: 221017-t-3 Danica>, <Slot: 221017-p-I Doug>, '...(remaining elements truncated)...']>

In [14]:
for slot in Slot.objects.all():
    slot.save()

In [15]:
for wd in Workday.objects.filter(iweek=44):
    print(wd,wd.week)

2022 10 30 None
2022 10 31 None
2022 11 01 None
2022 11 02 None
2022 11 03 None
2022 11 04 None
2022 11 05 None


In [15]:
n = Workday.objects.all().count()
print( f"TOTAL WORKDAYS: {n} " )

TOTAL WORKDAYS: 366 


In [16]:
from django import forms
import datetime as dt
class ArticleForm(forms.Form):
     title = forms.CharField()
     pub_date = forms.DateField()

In [17]:
""" 
Janet’s ducks lay 16 eggs per day. She eats three for breakfast 
every morning and bakes muffins for her friends every day with 
four. She sells the remainder for $2 per egg. How much does she
make every day? SHOW YOUR CALCULATIONS
"""

# Define the number of eggs Janet's ducks lay per day.
eggs_per_day = 16

# Define the number of eggs Janet eats for breakfast every morning.
eggs_for_breakfast = 3

# Define the number of eggs Janet uses to bake muffins for her friends every day.
eggs_for_muffins = 4

# Define the price Janet sells her eggs for.
eggs_price = 2

# Calculate the number of eggs Janet has left over after eating breakfast and baking muffins.
eggs_leftover = eggs_per_day - eggs_for_breakfast - eggs_for_muffins

# Calculate the amount of money Janet makes selling her eggs.
eggs_money = eggs_leftover * eggs_price

# Print the number of eggs Janet has left over after eating breakfast and baking muffins.
print("Janet has " + str(eggs_leftover) + " eggs left over after eating breakfast and baking muffins.")

# Print the amount of money Janet makes selling her eggs.
print("Janet makes $" + str(eggs_money) + " selling her eggs.")

Janet has 9 eggs left over after eating breakfast and baking muffins.
Janet makes $18 selling her eggs.


In [18]:
def turnarounds_withTrades ():
    tradeable = {}
    tas = Slot.objects.filter(is_turnaround=True)|Slot.objects.filter(is_preturnaround=True)
    slts = tas.filter(employee__evening_pref=True)
    empl_wantsThisSwapped = slts.filter(shift__start__hour__lt=10)
    for i in range(empl_wantsThisSwapped.count()):
        s = empl_wantsThisSwapped[i]
        if s.fillableBy().count() > 0:
            if not tradeable.get('pref'):
                tradeable['pref'] = []
            tradeable['pref'] += [s]
    empl_likesThisSlot = slts.filter(shift__start__hour__gte=10)
    for i in range(empl_likesThisSlot.count()):
        s = empl_likesThisSlot[i]
        if s.fillableBy().count() > 0:
            if not tradeable.get('nonpref'):
                tradeable['nonpref'] = []
            tradeable['nonpref'] += [s]
    return tradeable
    
x = turnarounds_withTrades()
x

{}

In [19]:
t = {}
a = t.get('a')
print(a)

None


In [16]:
def weekSlots_for_Empl (yearN, weekN, employee):
    slots = Slot.objects.filter(workday__iweek=weekN,workday__date__year=yearN,employee=employee)
    for i in range(7):
        s = slots.filter(workday__iweekday=i)
        if s.exists() == False:
            yield None
        else:
            yield s.first()
            
def display_weekSlots_for_Empl (yearN, weekN, employee):
    slots = [slot for slot in weekSlots_for_Empl(yearN,weekN,employee)]
    wds = 'SUN MON TUE WED THU FRI SAT'.split(' ')
    empl = f'{employee}'
    
    print(f'{yearN}-Week_{weekN} employee schedule:')
    print(f'_________ {empl} __________')
    for i in range(7):
        print(f'{wds[i]}: {slots[i]}')
    print('==========================')

In [17]:
slots = weekSlots_for_Empl(2022,44,Employee.objects.get(name='Josh'))
slots = [slot for slot in slots]
display_weekSlots_for_Empl(2022,44,Employee.objects.get(name='Molly'))
display_weekSlots_for_Empl(2022,44,Employee.objects.get(name='Josh'))

2022-Week_44 employee schedule:
_________ Molly __________
SUN: None
MON: 221031-p-I Molly
TUE: 221101-p-I Molly
WED: 221102-p-I Molly
THU: 221103-p-3 Molly
FRI: None
SAT: None
2022-Week_44 employee schedule:
_________ Josh __________
SUN: None
MON: None
TUE: 221101-t-S Josh
WED: 221102-t-MI Josh
THU: 221103-t-S Josh
FRI: None
SAT: None


In [18]:
def emp1_emp2_cowork_days (emp1,emp2,yearN,schN):
    jtsSlots = Slot.objects.filter(employee=emp1,workday__date__year=yearN,workday__ischedule=schN)
    nkiSlots = Slot.objects.filter(employee=emp2,workday__date__year=yearN,workday__ischedule=schN)
    dates = jtsSlots.union(nkiSlots)
    dates = dates.values('workday')
    for d in dates:
        if d in jtsSlots.values('workday'):
            if d in nkiSlots.values('workday'):
                yield d['workday']
                # "[207,217,223...]"
josh = Employee.objects.get(name="Josh")    
nicki = Employee.objects.get(name="Nicki")      
xs = emp1_emp2_cowork_days(josh,nicki,2022,5)
days = Workday.objects.filter(pk__in=xs)


print(f'COWORK DAYS FOR {josh} and {nicki}')
print("TOTAL FOR SCHEDULE:", days.count())
print(list(days.values_list('slug',flat=True)))



COWORK DAYS FOR Josh and Nicki
TOTAL FOR SCHEDULE: 7
['2022-07-29', '2022-08-09', '2022-08-11', '2022-08-19', '2022-08-25', '2022-09-01', '2022-09-02']


In [20]:
josh_slots_w_nicki = Slot.objects.filter(workday__in=days,employee=josh)
josh_slots_w_nicki 
for jslot in josh_slots_w_nicki:
    nslot = Slot.objects.get(workday=jslot.workday,employee=nicki)
    print(jslot,"  |  ",nslot)

220729-t-7C Josh   |   220729-t-EI Nicki
220809-t-S Josh   |   220809-t-EP Nicki
220811-t-S Josh   |   220811-t-MI Nicki
220819-t-7C Josh   |   220819-t-EI Nicki
220825-t-S Josh   |   220825-t-EI Nicki
220901-t-7C Josh   |   220901-t-EP Nicki
220902-t-S Josh   |   220902-t-EI Nicki


## Employee Training Comparisons

In [21]:
vals = Employee.objects.annotate(n_shifts_trained=Count('shifts_trained')).order_by('n_shifts_trained').values_list('name','n_shifts_trained')
for val in vals:
    print ("%s is trained for %s shift(s)." % (val[0], val[1]))

Brittanie is trained for 1 shift(s).
Brianna S is trained for 1 shift(s).
Amanda is trained for 1 shift(s).
Michael is trained for 1 shift(s).
Mary is trained for 1 shift(s).
Mark B is trained for 1 shift(s).
Bara is trained for 1 shift(s).
Corwin is trained for 1 shift(s).
Mei is trained for 2 shift(s).
Dianne is trained for 2 shift(s).
Mona is trained for 3 shift(s).
Tony is trained for 3 shift(s).
Erin is trained for 3 shift(s).
Brianna is trained for 4 shift(s).
Elisia is trained for 4 shift(s).
David is trained for 4 shift(s).
Nicki is trained for 4 shift(s).
Cheryl is trained for 4 shift(s).
Jaden is trained for 5 shift(s).
Leslie is trained for 5 shift(s).
Tiffany is trained for 6 shift(s).
Kat is trained for 6 shift(s).
Esperanza is trained for 7 shift(s).
Sam is trained for 7 shift(s).
Kristen is trained for 7 shift(s).
Josh is trained for 8 shift(s).
Sabrina is trained for 8 shift(s).
Molly is trained for 8 shift(s).
Doug is trained for 8 shift(s).
Carrie is trained for 8 shi